In [1]:
import random
import gym

env = gym.make("ALE/Asteroids-v5")

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]
/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
env.unwrapped.get_action_meanings()

['NOOP',
 'FIRE',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'UPFIRE',
 'RIGHTFIRE',
 'LEFTFIRE',
 'DOWNFIRE',
 'UPRIGHTFIRE',
 'UPLEFTFIRE']

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


In [4]:
states = env.observation_space.shape
actions = env.action_space.n

print(states)
print(actions)

(210, 160, 3)
14


In [5]:
model = Sequential()
model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,) + states))
model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(actions, activation='linear'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten (Flatten)           (None, 67584)             0         
                                                                 
 dense (Dense)               (None, 512)               34603520  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 14)                3

In [7]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [10]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
memory = SequentialMemory(limit=1000, window_length=3)

agent = DQNAgent(
  model=model,
  memory=memory,
  policy=policy,
  enable_dueling_network=True,
  dueling_type='avg',
  nb_actions=actions,
  nb_steps_warmup=1000
)

agent.compile(Adam(lr=0.001), metrics=['mae'])

/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
import os

iterations = 20
steps_per_iteration = 100000
base_dir = '/mnt/c/users/haval/OneDrive - Oregon Institute of Technology/Documents/CST407/project/car_racing'

for i in range(1, iterations + 1):
  agent.fit(env, nb_steps=steps_per_iteration, visualize=False, verbose=1)
  
  folder_name = f'cart_pole_{i * steps_per_iteration}'
  folder_path = os.path.join(base_dir, folder_name)
  os.makedirs(folder_path, exist_ok=True)

  save_path = os.path.join(folder_path, 'weights.h5f')
  agent.save_weights(save_path)
  print(f'Saved weights to {save_path}')

Training for 100000 steps ...
Interval 1 (0 steps performed)


/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   13/10000 [..............................] - ETA: 2:22 - reward: 0.0000e+00

/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


  329/10000 [..............................] - ETA: 2:19 - reward: 0.6383done, took 5.022 seconds
Saved weights to /mnt/c/users/haval/OneDrive - Oregon Institute of Technology/Documents/CST407/project/car_racing/cart_pole_100000/weights.h5f
Training for 100000 steps ...
Interval 1 (0 steps performed)
  726/10000 [=>............................] - ETA: 2:20 - reward: 1.0055

In [17]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

/home/doolyboi/miniconda3/envs/tf/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [20]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
  449/10000 [>.............................] - ETA: 2:56 - reward: 1.8486done, took 8.431 seconds
